In [1]:
import pandas as pd
import numpy as np
#import openpyxl
import os
from tqdm import tqdm
tqdm.pandas()

import tensorflow as tf


from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
import tensorflow_decision_forests as tfdf

2024-03-11 11:32:19.804532: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-11 11:32:19.840733: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-11 11:32:19.840760: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-11 11:32:19.841477: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-11 11:32:19.846961: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-11 11:32:19.847620: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [5]:
model = tf.keras.models.load_model("../data/6_models/Elo_1500_1600.csv.pkl")

[INFO 24-03-11 11:33:59.9978 WET kernel.cc:1233] Loading model from path ../data/6_models/Elo_1500_1600.csv.pkl/assets/ with prefix 36378ba4dc25402a
[INFO 24-03-11 11:34:03.2378 WET decision_forest.cc:660] Model loaded with 300 root(s), 2335076 node(s), and 30 input feature(s).
[INFO 24-03-11 11:34:03.2379 WET abstract_model.cc:1344] Engine "RandomForestOptPred" built
[INFO 24-03-11 11:34:03.2379 WET kernel.cc:1061] Use fast generic engine


In [6]:
model.summary()

Model: "random_forest_model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
Total params: 1 (1.00 Byte)
Trainable params: 0 (0.00 Byte)
Non-trainable params: 1 (1.00 Byte)
_________________________________________________________________


In [85]:
df = pd.read_pickle("../data/5_lichess_csv_ML_ready/Elo_1500_1600.csv.pkl")

df = df.drop(columns=['Board','MoveNumber','WhiteElo','BlackElo'])
df = df.reset_index(drop=True)
df = df.reindex(sorted(df.columns), axis=1)

# remove draws for now
#df = df[df['Result'] != 2]

In [86]:
df.columns.tolist()

['B_white',
 'Doubled_pawns_black',
 'Doubled_pawns_white',
 'Isolated_pawns_black',
 'Isolated_pawns_white',
 'K_white',
 'King_pawn_shield_black',
 'King_pawn_shield_white',
 'King_zone_attacked_squares_black',
 'King_zone_attacked_squares_white',
 'King_zone_controlled_squares_black',
 'King_zone_controlled_squares_white',
 'Mobility_black',
 'Mobility_white',
 'N_white',
 'P_white',
 'Passed_pawns_black',
 'Passed_pawns_white',
 'Q_white',
 'R_white',
 'Result',
 'Trapped_bishops_black',
 'Trapped_bishops_white',
 'Trapped_queens_black',
 'Trapped_queens_white',
 'Trapped_rooks_black',
 'Trapped_rooks_white',
 'b_black',
 'k_black',
 'n_black',
 'p_black',
 'q_black',
 'r_black']

In [87]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train_df, label="Result")

test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(test_df, label="Result")


In [88]:
#model.compile(metrics=["accuracy"])
#print(model.evaluate(test_ds))

In [90]:
test_ds

<_PrefetchDataset element_spec=({'B_white': TensorSpec(shape=(None,), dtype=tf.int8, name=None), 'Doubled_pawns_black': TensorSpec(shape=(None,), dtype=tf.int8, name=None), 'Doubled_pawns_white': TensorSpec(shape=(None,), dtype=tf.int8, name=None), 'Isolated_pawns_black': TensorSpec(shape=(None,), dtype=tf.int8, name=None), 'Isolated_pawns_white': TensorSpec(shape=(None,), dtype=tf.int8, name=None), 'K_white': TensorSpec(shape=(None,), dtype=tf.int8, name=None), 'King_pawn_shield_black': TensorSpec(shape=(None,), dtype=tf.int8, name=None), 'King_pawn_shield_white': TensorSpec(shape=(None,), dtype=tf.int8, name=None), 'King_zone_attacked_squares_black': TensorSpec(shape=(None,), dtype=tf.int8, name=None), 'King_zone_attacked_squares_white': TensorSpec(shape=(None,), dtype=tf.int8, name=None), 'King_zone_controlled_squares_black': TensorSpec(shape=(None,), dtype=tf.int8, name=None), 'King_zone_controlled_squares_white': TensorSpec(shape=(None,), dtype=tf.int8, name=None), 'Mobility_black

In [80]:
# use the model to make predictions
predictions = model.predict(test_ds)

59/59 [==============================] - 4s 68ms/step


In [27]:
predictions

array([[0.12999995],
       [0.93333256],
       [0.02      ],
       ...,
       [0.54666626],
       [0.24333318],
       [0.2133332 ]], dtype=float32)

In [34]:
teste = ['Result', 'SideToMove', 'white_kingside_castling_rights', 'white_queenside_castling_rights', 'black_kingside_castling_rights', 'black_queenside_castling_rights', 'K_white', 'Q_white', 'R_white', 'B_white', 'N_white', 'P_white', 'k_black', 'q_black', 'r_black', 'b_black', 'n_black', 'p_black', '0_white_type', '0_white_row', '0_white_col', '0_white', '0_white_min_attacker', '0_white_min_defender', '0_black_type', '0_black_row', '0_black_col', '0_black', '0_black_min_attacker', '0_black_min_defender', '1_white_type', '1_white_row', '1_white_col', '1_white', '1_white_min_attacker', '1_white_min_defender', '1_black_type', '1_black_row', '1_black_col', '1_black', '1_black_min_attacker', '1_black_min_defender', '2_white_type', '2_white_row', '2_white_col', '2_white', '2_white_min_attacker', '2_white_min_defender', '2_black_type', '2_black_row', '2_black_col', '2_black', '2_black_min_attacker', '2_black_min_defender', '3_white_type', '3_white_row', '3_white_col', '3_white', '3_white_min_attacker', '3_white_min_defender', '3_black_type', '3_black_row', '3_black_col', '3_black', '3_black_min_attacker', '3_black_min_defender', '4_white_type', '4_white_row', '4_white_col', '4_white', '4_white_min_attacker', '4_white_min_defender', '4_black_type', '4_black_row', '4_black_col', '4_black', '4_black_min_attacker', '4_black_min_defender', '5_white_type', '5_white_row', '5_white_col', '5_white', '5_white_min_attacker', '5_white_min_defender', '5_black_type', '5_black_row', '5_black_col', '5_black', '5_black_min_attacker', '5_black_min_defender', '6_white_type', '6_white_row', '6_white_col', '6_white', '6_white_min_attacker', '6_white_min_defender', '6_black_type', '6_black_row', '6_black_col', '6_black', '6_black_min_attacker', '6_black_min_defender', '7_white_type', '7_white_row', '7_white_col', '7_white', '7_white_min_attacker', '7_white_min_defender', '7_black_type', '7_black_row', '7_black_col', '7_black', '7_black_min_attacker', '7_black_min_defender', '8_white_type', '8_white_row', '8_white_col', '8_white', '8_white_min_attacker', '8_white_min_defender', '8_black_type', '8_black_row', '8_black_col', '8_black', '8_black_min_attacker', '8_black_min_defender', '9_white_type', '9_white_row', '9_white_col', '9_white', '9_white_min_attacker', '9_white_min_defender', '9_black_type', '9_black_row', '9_black_col', '9_black', '9_black_min_attacker', '9_black_min_defender', '10_white_type', '10_white_row', '10_white_col', '10_white', '10_white_min_attacker', '10_white_min_defender', '10_black_type', '10_black_row', '10_black_col', '10_black', '10_black_min_attacker', '10_black_min_defender', '11_white_type', '11_white_row', '11_white_col', '11_white', '11_white_min_attacker', '11_white_min_defender', '11_black_type', '11_black_row', '11_black_col', '11_black', '11_black_min_attacker', '11_black_min_defender', '12_white_type', '12_white_row', '12_white_col', '12_white', '12_white_min_attacker', '12_white_min_defender', '12_black_type', '12_black_row', '12_black_col', '12_black', '12_black_min_attacker', '12_black_min_defender', '13_white_type', '13_white_row', '13_white_col', '13_white', '13_white_min_attacker', '13_white_min_defender', '13_black_type', '13_black_row', '13_black_col', '13_black', '13_black_min_attacker', '13_black_min_defender', '14_white_type', '14_white_row', '14_white_col', '14_white', '14_white_min_attacker', '14_white_min_defender', '14_black_type', '14_black_row', '14_black_col', '14_black', '14_black_min_attacker', '14_black_min_defender', '15_white_type', '15_white_row', '15_white_col', '15_white', '15_white_min_attacker', '15_white_min_defender', '15_black_type', '15_black_row', '15_black_col', '15_black', '15_black_min_attacker', '15_black_min_defender', '0_white_D0', '0_white_D1', '0_white_D2', '0_white_D3', '0_white_D4', '0_white_D5', '0_white_D6', '0_white_D7', '0_black_D0', '0_black_D1', '0_black_D2', '0_black_D3', '0_black_D4', '0_black_D5', '0_black_D6', '0_black_D7', '1_white_D0', '1_white_D1', '1_white_D2', '1_white_D3', '1_white_D4', '1_white_D5', '1_white_D6', '1_white_D7', '1_black_D0', '1_black_D1', '1_black_D2', '1_black_D3', '1_black_D4', '1_black_D5', '1_black_D6', '1_black_D7', '2_white_D0', '2_white_D1', '2_white_D2', '2_white_D3', '2_white_D4', '2_white_D5', '2_white_D6', '2_white_D7', '2_black_D0', '2_black_D1', '2_black_D2', '2_black_D3', '2_black_D4', '2_black_D5', '2_black_D6', '2_black_D7', '3_white_D0', '3_white_D1', '3_white_D2', '3_white_D3', '3_white_D4', '3_white_D5', '3_white_D6', '3_white_D7', '3_black_D0', '3_black_D1', '3_black_D2', '3_black_D3', '3_black_D4', '3_black_D5', '3_black_D6', '3_black_D7', '4_white_D0', '4_white_D1', '4_white_D2', '4_white_D3', '4_white_D4', '4_white_D5', '4_white_D6', '4_white_D7', '4_black_D0', '4_black_D1', '4_black_D2', '4_black_D3', '4_black_D4', '4_black_D5', '4_black_D6', '4_black_D7', '5_white_D0', '5_white_D1', '5_white_D2', '5_white_D3', '5_white_D4', '5_white_D5', '5_white_D6', '5_white_D7', '5_black_D0', '5_black_D1', '5_black_D2', '5_black_D3', '5_black_D4', '5_black_D5', '5_black_D6', '5_black_D7', '6_white_D0', '6_white_D1', '6_white_D2', '6_white_D3', '6_white_D4', '6_white_D5', '6_white_D6', '6_white_D7', '6_black_D0', '6_black_D1', '6_black_D2', '6_black_D3', '6_black_D4', '6_black_D5', '6_black_D6', '6_black_D7', '7_white_D0', '7_white_D1', '7_white_D2', '7_white_D3', '7_white_D4', '7_white_D5', '7_white_D6', '7_white_D7', '7_black_D0', '7_black_D1', '7_black_D2', '7_black_D3', '7_black_D4', '7_black_D5', '7_black_D6', '7_black_D7', '8_white_D0', '8_white_D1', '8_white_D2', '8_white_D3', '8_white_D4', '8_white_D5', '8_white_D6', '8_white_D7', '8_black_D0', '8_black_D1', '8_black_D2', '8_black_D3', '8_black_D4', '8_black_D5', '8_black_D6', '8_black_D7', '9_white_D0', '9_white_D1', '9_white_D2', '9_white_D3', '9_white_D4', '9_white_D5', '9_white_D6', '9_white_D7', '9_black_D0', '9_black_D1', '9_black_D2', '9_black_D3', '9_black_D4', '9_black_D5', '9_black_D6', '9_black_D7', '10_white_D0', '10_white_D1', '10_white_D2', '10_white_D3', '10_white_D4', '10_white_D5', '10_white_D6', '10_white_D7', '10_black_D0', '10_black_D1', '10_black_D2', '10_black_D3', '10_black_D4', '10_black_D5', '10_black_D6', '10_black_D7', '11_white_D0', '11_white_D1', '11_white_D2', '11_white_D3', '11_white_D4', '11_white_D5', '11_white_D6', '11_white_D7', '11_black_D0', '11_black_D1', '11_black_D2', '11_black_D3', '11_black_D4', '11_black_D5', '11_black_D6', '11_black_D7', '12_white_D0', '12_white_D1', '12_white_D2', '12_white_D3', '12_white_D4', '12_white_D5', '12_white_D6', '12_white_D7', '12_black_D0', '12_black_D1', '12_black_D2', '12_black_D3', '12_black_D4', '12_black_D5', '12_black_D6', '12_black_D7', '13_white_D0', '13_white_D1', '13_white_D2', '13_white_D3', '13_white_D4', '13_white_D5', '13_white_D6', '13_white_D7', '13_black_D0', '13_black_D1', '13_black_D2', '13_black_D3', '13_black_D4', '13_black_D5', '13_black_D6', '13_black_D7', '14_white_D0', '14_white_D1', '14_white_D2', '14_white_D3', '14_white_D4', '14_white_D5', '14_white_D6', '14_white_D7', '14_black_D0', '14_black_D1', '14_black_D2', '14_black_D3', '14_black_D4', '14_black_D5', '14_black_D6', '14_black_D7', '15_white_D0', '15_white_D1', '15_white_D2', '15_white_D3', '15_white_D4', '15_white_D5', '15_white_D6', '15_white_D7', '15_black_D0', '15_black_D1', '15_black_D2', '15_black_D3', '15_black_D4', '15_black_D5', '15_black_D6', '15_black_D7', '0_0_min_attacker', '0_0_min_defender', '0_1_min_attacker', '0_1_min_defender', '0_2_min_attacker', '0_2_min_defender', '0_3_min_attacker', '0_3_min_defender', '0_4_min_attacker', '0_4_min_defender', '0_5_min_attacker', '0_5_min_defender', '0_6_min_attacker', '0_6_min_defender', '0_7_min_attacker', '0_7_min_defender', '1_0_min_attacker', '1_0_min_defender', '1_1_min_attacker', '1_1_min_defender', '1_2_min_attacker', '1_2_min_defender', '1_3_min_attacker', '1_3_min_defender', '1_4_min_attacker', '1_4_min_defender', '1_5_min_attacker', '1_5_min_defender', '1_6_min_attacker', '1_6_min_defender', '1_7_min_attacker', '1_7_min_defender', '2_0_min_attacker', '2_0_min_defender', '2_1_min_attacker', '2_1_min_defender', '2_2_min_attacker', '2_2_min_defender', '2_3_min_attacker', '2_3_min_defender', '2_4_min_attacker', '2_4_min_defender', '2_5_min_attacker', '2_5_min_defender', '2_6_min_attacker', '2_6_min_defender', '2_7_min_attacker', '2_7_min_defender', '3_0_min_attacker', '3_0_min_defender', '3_1_min_attacker', '3_1_min_defender', '3_2_min_attacker', '3_2_min_defender', '3_3_min_attacker', '3_3_min_defender', '3_4_min_attacker', '3_4_min_defender', '3_5_min_attacker', '3_5_min_defender', '3_6_min_attacker', '3_6_min_defender', '3_7_min_attacker', '3_7_min_defender', '4_0_min_attacker', '4_0_min_defender', '4_1_min_attacker', '4_1_min_defender', '4_2_min_attacker', '4_2_min_defender', '4_3_min_attacker', '4_3_min_defender', '4_4_min_attacker', '4_4_min_defender', '4_5_min_attacker', '4_5_min_defender', '4_6_min_attacker', '4_6_min_defender', '4_7_min_attacker', '4_7_min_defender', '5_0_min_attacker', '5_0_min_defender', '5_1_min_attacker', '5_1_min_defender', '5_2_min_attacker', '5_2_min_defender', '5_3_min_attacker', '5_3_min_defender', '5_4_min_attacker', '5_4_min_defender', '5_5_min_attacker', '5_5_min_defender', '5_6_min_attacker', '5_6_min_defender', '5_7_min_attacker', '5_7_min_defender', '6_0_min_attacker', '6_0_min_defender', '6_1_min_attacker', '6_1_min_defender', '6_2_min_attacker', '6_2_min_defender', '6_3_min_attacker', '6_3_min_defender', '6_4_min_attacker', '6_4_min_defender', '6_5_min_attacker', '6_5_min_defender', '6_6_min_attacker', '6_6_min_defender', '6_7_min_attacker', '6_7_min_defender', '7_0_min_attacker', '7_0_min_defender', '7_1_min_attacker', '7_1_min_defender', '7_2_min_attacker', '7_2_min_defender', '7_3_min_attacker', '7_3_min_defender', '7_4_min_attacker', '7_4_min_defender', '7_5_min_attacker', '7_5_min_defender', '7_6_min_attacker', '7_6_min_defender', '7_7_min_attacker', '7_7_min_defender']

In [45]:
test_df.columns.to_list()[18]

'0_white_type'

In [14]:
model.predict(teste)

ValueError: in user code:

    File "/home/pedro/.local/lib/python3.10/site-packages/keras/src/engine/training.py", line 2440, in predict_function  *
        return step_function(self, iterator)
    File "/home/pedro/.local/lib/python3.10/site-packages/keras/src/engine/training.py", line 2425, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/pedro/.local/lib/python3.10/site-packages/keras/src/engine/training.py", line 2413, in run_step  **
        outputs = model.predict_step(data)
    File "/home/pedro/.local/lib/python3.10/site-packages/keras/src/engine/training.py", line 2381, in predict_step
        return self(x, training=False)
    File "/home/pedro/.local/lib/python3.10/site-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None

    ValueError: Could not find matching concrete function to call loaded from the SavedModel. Got:
      Positional argume